In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy import stats
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn import metrics
import scipy

In [3]:
fmli = pd.read_csv("fmli_15_clean.csv")
fmli.drop(["DETTACHED"], axis = 1, inplace = True)

##### Normalize data frame

In [4]:
# Log every variables, save it in a new data frame for future use.
log = np.log(fmli)
log["OWN"] = fmli["OWN"]
log["DEGREE"] = fmli["DEGREE"]
log["VEHICLES"] = fmli["VEHICLES"]

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log
  


##### Multivariable Model: HHSIZE + INCOME + ROOMS  + OWN + DEGREE + VEH

In [33]:
N = []
adj_r = []
b_hhsize = []
b_income = []
b_rooms = []
b_own = []
b_degree = []
b_veh = []
se_hhsize = []
se_income = []
se_rooms = []
se_own = []
se_degree = []
se_veh = []
p_hhsize = []
p_income = []
p_rooms = []
p_own = []
p_degree = []
p_veh = []

for i in range(16, 17):
    df = log[["HHSIZE", "INCOME", "ROOMS", "OWN", "DEGREE", "VEHICLES", log.columns[i]]]
    df.replace(np.Inf, np.nan, inplace = True)
    df.replace(-np.Inf, np.nan, inplace = True)
    df.dropna(inplace = True)
    df.reset_index(inplace = True)
    X = df[["HHSIZE", "INCOME", "ROOMS", "OWN", "DEGREE", "VEHICLES"]]
    Y = df[log.columns[i]]
    X1 = sm.add_constant(X)
    model = sm.OLS(Y, X1).fit()
    N.append(len(Y))
    b_hhsize.append(model.params[1])
    b_income.append(model.params[2])
    b_rooms.append(model.params[3])
    b_own.append(model.params[4])
    b_degree.append(model.params[5])
    b_veh.append(model.params[6])
    adj_r.append(model.rsquared_adj)
    se_hhsize.append(model.bse[1])
    se_income.append(model.bse[2])
    se_rooms.append(model.bse[3])
    se_own.append(model.bse[4])
    se_degree.append(model.bse[5])
    se_veh.append(model.bse[6])
    p_hhsize.append(model.pvalues[1])
    p_income.append(model.pvalues[2])
    p_rooms.append(model.pvalues[3])
    p_own.append(model.pvalues[4])
    p_degree.append(model.pvalues[5])
    p_veh.append(model.pvalues[6])

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


##### Second model with Lasso dropping insignificant variables

In [5]:
N = []
adj_r = []
b_hhsize = []
b_income = []
b_rooms = []
b_own = []
b_degree = []
b_veh = []
se_hhsize = []
se_income = []
se_rooms = []
se_own = []
se_degree = []
se_veh = []
p_hhsize = []
p_income = []
p_rooms = []
p_own = []
p_degree = []
p_veh = []

for i in range(16, len(log.columns)):
    df = log[["HHSIZE", "INCOME", "ROOMS", "OWN", "DEGREE", "VEHICLES", log.columns[i]]]
    df.replace(np.Inf, np.nan, inplace = True)
    df.replace(-np.Inf, np.nan, inplace = True)
    df.dropna(inplace = True)
    df.reset_index(inplace = True)
    X = df[["HHSIZE", "INCOME", "ROOMS", "OWN", "DEGREE", "VEHICLES"]]
    Y = df[log.columns[i]]
    X1 = sm.add_constant(X)
    model = linear_model.Lasso(alpha=0.01).fit(X1, Y)
    N.append(len(Y))
    b_hhsize.append(model.coef_[1])
    b_income.append(model.coef_[2])
    b_rooms.append(model.coef_[3])
    b_own.append(model.coef_[4])
    b_degree.append(model.coef_[5])
    b_veh.append(model.coef_[6])
    

    new = pd.DataFrame()
    if model.coef_[1] != 0:
        new["HHSIZE"] = df["HHSIZE"]
    if model.coef_[2] != 0:
        new["INCOME"] = df["INCOME"]
    if model.coef_[3] != 0:
        new["ROOMS"] = df["ROOMS"]
    if model.coef_[4] != 0:
        new["OWN"] = df["OWN"]
    if model.coef_[5] != 0:
        new["DEGREE"] = df["DEGREE"]
    if model.coef_[6] != 0:
        new["VEHICLES"] = df["VEHICLES"]
    new = sm.add_constant(new)
    model1 = sm.OLS(Y, new).fit() 
    adj_r.append(model1.rsquared_adj)
    bse = list(model1.bse)
    bse.pop(0)
    p = list(model1.pvalues)
    p.pop(0)
    if model.coef_[1] != 0:
        se_hhsize.append(bse.pop(0))
        p_hhsize.append(p.pop(0))
    else:
        se_hhsize.append(-1)
        p_hhsize.append(-1)
    if model.coef_[2] != 0:
        se_income.append(bse.pop(0))
        p_income.append(p.pop(0))
    else:
        se_income.append(-1)
        p_income.append(-1)
    if model.coef_[3] != 0:
        se_rooms.append(bse.pop(0))
        p_rooms.append(p.pop(0))
    else:
        se_rooms.append(-1)
        p_rooms.append(-1)
    if model.coef_[4] != 0:
        se_own.append(bse.pop(0))
        p_own.append(p.pop(0))
    else:
        se_own.append(-1)
        p_own.append(-1)
    if model.coef_[5] != 0:
        se_degree.append(bse.pop(0))
        p_degree.append(p.pop(0))
    else:
        se_degree.append(-1)
        p_degree.append(-1)
    if model.coef_[6] != 0:
        se_veh.append(bse.pop(0))
        p_veh.append(p.pop(0))
    else:
        se_veh.append(-1)
        p_veh.append(-1)

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and w

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and w

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be se

In [6]:
model.coef_

array([0.        , 0.22337458, 0.23889616, 0.18646872, 0.        ,
       0.25955218, 0.07558012])

In [7]:
b_hhsize = [round(x, 3) for x in b_hhsize]
b_income = [round(x, 3) for x in b_income]
b_rooms = [round(x, 3) for x in b_rooms]
b_own = [round(x, 3) for x in b_own]
b_degree = [round(x, 3) for x in b_degree]
b_veh = [round(x, 3) for x in b_veh]

se_hhsize = [round(x, 3) for x in se_hhsize]
se_income = [round(x, 3) for x in se_income]
se_rooms = [round(x, 3) for x in se_rooms]
se_own = [round(x, 3) for x in se_own]
se_degree = [round(x, 3) for x in se_degree]
se_veh = [round(x, 3) for x in se_veh]

p_hhsize = [round(x, 3) for x in p_hhsize]
p_income = [round(x, 3) for x in p_income]
p_rooms = [round(x, 3) for x in p_rooms]
p_own = [round(x, 3) for x in p_own]
p_degree = [round(x, 3) for x in p_degree]
p_veh = [round(x, 3) for x in p_veh]

N = [round(x, 0) for x in N]
adj_r = [round(x, 3) for x in adj_r]

In [8]:
index = []
for i in range(len(log.columns[16: len(log.columns)].values)):
    index.append(log.columns[16: len(log.columns)].values[i])
for i in range(len(log.columns[16: len(log.columns)].values)):
    index.append(log.columns[16: len(log.columns)].values[i])
for i in range(len(log.columns[16: len(log.columns)].values)):
    index.append(log.columns[16: len(log.columns)].values[i])

In [9]:
for i in range(len(b_hhsize)):
    if b_hhsize[i] == 0:
        b_hhsize[i] = " "
for i in range(len(b_income)):
    if b_income[i] == 0:
        b_income[i] = " "
for i in range(len(b_rooms)):
    if b_rooms[i] == 0:
        b_rooms[i] = " "
for i in range(len(b_own)):
    if b_own[i] == 0:
        b_own[i] = " "
for i in range(len(b_degree)):
    if b_degree[i] == 0:
        b_degree[i] = " "
for i in range(len(b_veh)):
    if b_veh[i] == 0:
        b_veh[i] = " "
        
for i in range(len(se_hhsize)):
    if se_hhsize[i] < 0:
        se_hhsize[i] = " "
for i in range(len(se_income)):
    if se_income[i] < 0:
        se_income[i] = " "
for i in range(len(se_rooms)):
    if se_rooms[i] < 0:
        se_rooms[i] = " "
for i in range(len(se_own)):
    if se_own[i] < 0:
        se_own[i] = " "
for i in range(len(se_degree)):
    if se_degree[i] < 0:
        se_degree[i] = " "
for i in range(len(se_veh)):
    if se_veh[i] < 0:
        se_veh[i] = " "
        
for i in range(len(p_hhsize)):
    if p_hhsize[i] < 0:
        p_hhsize[i] = " "
for i in range(len(p_income)):
    if p_income[i] < 0:
        p_income[i] = " "
for i in range(len(p_rooms)):
    if p_rooms[i] < 0:
        p_rooms[i] = " "
for i in range(len(p_own)):
    if p_own[i] < 0:
        p_own[i] = " "
for i in range(len(p_degree)):
    if p_degree[i] < 0:
        p_degree[i] = " "
for i in range(len(p_veh)):
    if p_veh[i] < 0:
        p_veh[i] = " "

In [10]:
len(p_degree)

54

In [11]:
parameter = pd.DataFrame({"parameter": ["Beta"] * 54 + ["SE"] * 54 + ["Confidence Level"] * 54,
                          "HHSIZE": np.concatenate([b_hhsize, se_hhsize, p_hhsize]),
                          "INCOME": np.concatenate([b_income, se_income, p_income]),
                          "ROOMS": np.concatenate([b_rooms, se_rooms, p_rooms]),
                          "OWN": np.concatenate([b_own, se_own, p_own]),
                          "DEGREE": np.concatenate([b_degree, se_degree, p_degree]),
                          "VEH": np.concatenate([b_veh, se_veh, p_veh]),
                          "N": N + [" "] * 54 + ["  "] * 54,
                          "adj_r2": adj_r + [" "] * 54 + ["  "] * 54,
                          "index": index})

In [12]:
parameter = pd.DataFrame(parameter.groupby(["index", "N", "adj_r2", "parameter", "HHSIZE", "INCOME", "ROOMS", "OWN", "DEGREE", "VEH"]).apply(list)).drop([0], axis = 1)

In [13]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(parameter)

Empty DataFrame
Columns: []
Index: [(AdjustedTotalExpenditures, 26855, 0.36, Beta, 0.223, 0.239, 0.186,  , 0.26, 0.076), (AdjustedTotalExpenditures,  ,  , SE, 0.008, 0.004, 0.012,  , 0.009, 0.003), (AdjustedTotalExpenditures,   ,   , Confidence Level, 0.0, 0.0, 0.0,  , 0.0, 0.0), (AlcoholicBeverages, 12960, 0.061, Beta, -0.14, 0.202, 0.035,  , 0.179, 0.04), (AlcoholicBeverages,  ,  , SE, 0.022, 0.01, 0.03,  , 0.023, 0.007), (AlcoholicBeverages,   ,   , Confidence Level, 0.0, 0.0, 0.0,  , 0.0, 0.0), (ApparelAndServices, 16365, 0.12, Beta, 0.435, 0.193, 0.167,  , 0.125, 0.013), (ApparelAndServices,  ,  , SE, 0.019, 0.009, 0.027,  , 0.022, 0.007), (ApparelAndServices,   ,   , Confidence Level, 0.0, 0.0, 0.0,  , 0.0, 0.155), (BabysittingAndChildDayCare, 1716, 0.062, Beta, -0.113, 0.219,  , 0.05, 0.193, -0.034), (BabysittingAndChildDayCare,  ,  , SE, 0.08, 0.029,  , 0.065, 0.063, 0.024), (BabysittingAndChildDayCare,   ,   , Confidence Level, 0.016, 0.0,  , 0.138, 0.0, 0.078), (CarsAndTrucksNewNetOutlay, 285, 0.043, Beta, 0.006, 0.035, 0.141,  , -0.051, 0.012), (CarsAndTrucksNewNetOutlay,  ,  , SE, 0.061, 0.026, 0.084,  , 0.059, 0.018), (CarsAndTrucksNewNetOutlay,   ,   , Confidence Level, 0.73, 0.132, 0.009,  , 0.077, 0.716), (CarsAndTrucksUsedNetOutlay, 746, 0.068, Beta, -0.05, 0.257, 0.231, 0.13, 0.169, -0.054), (CarsAndTrucksUsedNetOutlay,  ,  , SE, 0.096, 0.052, 0.148, 0.117, 0.103, 0.029), (CarsAndTrucksUsedNetOutlay,   ,   , Confidence Level, 0.33, 0.0, 0.041, 0.18, 0.057, 0.03), (CashContributions, 12474, 0.097, Beta, -0.075, 0.216, 0.427, 0.189, 0.22, 0.049), (CashContributions,  ,  , SE, 0.027, 0.012, 0.042, 0.035, 0.028, 0.009), (CashContributions,   ,   , Confidence Level, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), (DomesticServices, 7317, 0.184, Beta, 0.469, 0.212, 0.895, 0.119, 0.26, -0.046), (DomesticServices,  ,  , SE, 0.036, 0.016, 0.055, 0.05, 0.039, 0.013), (DomesticServices,   ,   , Confidence Level, 0.0, 0.0, 0.0, 0.001, 0.0, 0.0), (DomesticServicesExclChildcare, 6112, 0.222, Beta, -0.135, 0.18, 1.102, 0.472, 0.177, -0.016), (DomesticServicesExclChildcare,  ,  , SE, 0.036, 0.015, 0.054, 0.051, 0.038, 0.013), (DomesticServicesExclChildcare,   ,   , Confidence Level, 0.0, 0.0, 0.0, 0.0, 0.0, 0.059), (Education, 2609, 0.081, Beta, -0.446, 0.206, 0.261, 0.128, 0.376, 0.082), (Education,  ,  , SE, 0.067, 0.031, 0.101, 0.086, 0.071, 0.023), (Education,   ,   , Confidence Level, 0.0, 0.0, 0.001, 0.079, 0.0, 0.0), (Entertainment, 23419, 0.134, Beta, 0.104, 0.175, 0.272, 0.109, 0.232, 0.053), (Entertainment,  ,  , SE, 0.014, 0.006, 0.022, 0.018, 0.015, 0.005), (Entertainment,   ,   , Confidence Level, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), (FeesAndAdmissions, 9453, 0.124, Beta, 0.245, 0.188, 0.185, 0.177, 0.396,  ), (FeesAndAdmissions,  ,  , SE, 0.026, 0.012, 0.04, 0.034, 0.027, 0.009), (FeesAndAdmissions,   ,   , Confidence Level, 0.0, 0.0, 0.0, 0.0, 0.0, 0.762), (FloorCoverings, 822, 0.088, Beta, 0.042, 0.092, 0.288, 0.863, 0.046,  ), (FloorCoverings,  ,  , SE, 0.114, 0.056, 0.184, 0.159, 0.122,  ), (FloorCoverings,   ,   , Confidence Level, 0.542, 0.14, 0.082, 0.0, 0.493,  ), (FoodAtHome, 26603, 0.229, Beta, 0.517, 0.075, 0.065, 0.041, 0.05, 0.007), (FoodAtHome,  ,  , SE, 0.008, 0.004, 0.013, 0.011, 0.009, 0.003), (FoodAtHome,   ,   , Confidence Level, 0.0, 0.0, 0.0, 0.0, 0.0, 0.376), (FoodAwayFromHome, 21720, 0.119, Beta, 0.158, 0.19, 0.082, 0.01, 0.205, 0.023), (FoodAwayFromHome,  ,  , SE, 0.013, 0.006, 0.021, 0.017, 0.014, 0.005), (FoodAwayFromHome,   ,   , Confidence Level, 0.0, 0.0, 0.0, 0.028, 0.0, 0.0), (Furniture, 2409, 0.041, Beta,  , 0.181, 0.013, 0.303,  , -0.009), (Furniture,  ,  , SE,  , 0.028, 0.091, 0.077,  , 0.021), (Furniture,   ,   , Confidence Level,  , 0.0, 0.452, 0.0,  , 0.301), (HealthInsurance, 19190, 0.081, Beta, 0.212, 0.063, 0.187, 0.218, 0.031, 0.023), (HealthInsurance,  ,  , SE, 0.015, 0.007, 0.023, 0.019, 0.015, 0.005), (HealthInsurance,   ,   , Confidence Level, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), (Healthcare, 21

In [14]:
parameter.to_csv("multimodel_7var.csv")